In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin


def get_all_sublinks(url, max_depth=None, visited=None):
    """
    Recursively retrieve all sublinks from a webpage.

    Args:
        url (str): The starting URL.
        max_depth (int, optional): Maximum depth of recursion. None means no limit.
        visited (set, optional): Set of visited URLs to avoid duplicates.

    Returns:
        set: A set of all sublinks found.
    """
    if visited is None:
        visited = set()

    if max_depth is not None and max_depth <= 0:
        return set()

    try:
        response = requests.get(url)
        if response.status_code == 200:
            visited.add(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            sublinks = set()

            for link in soup.find_all('a', href=True):
                absolute_link = urljoin(url, link['href'])
                if absolute_link not in visited:
                    sublinks.add(absolute_link)
                    sublinks.update(get_all_sublinks(absolute_link, max_depth - 1, visited))

            return sublinks
    except Exception as e:
        print(f"An error occurred: {str(e)}")


# Example usage:
start_url = 'http://nobullconstructions.com.au'  # Replace with your desired starting URL
all_sublinks = get_all_sublinks(start_url, max_depth=1)  # Adjust the maximum depth as needed


In [ ]:
requests.get('http://www.nobullconstructions.com.au').status_code

In [ ]:
len(urls)

In [3]:
import requests
import json
import re
import pandas as pd
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from urllib.parse import urljoin
import xlsxwriter

# set headers
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.5',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1'
}
all_data = []

# import website and company name from excel file
urls = pd.read_excel('yellowpages5.xlsx')
urls = urls[['name','website']]
urls['text'] = ''
urls['sublinks'] = ''
urls['sublinkstext'] = ''
# scrape website text and store in dataframe
# something wrong with index 958
with requests.Session() as s:
            temp = 'grantleigh.com.au/'
            r = s.get('https://www.grantleigh.com.au/', verify=False, headers=headers)
            soup = BeautifulSoup(r.text, 'html.parser')
            text = soup.get_text()
            text = text.replace('\n', '').replace('\t', '')
            sublinks=[]
            for link in soup.find_all('a', href=True):
                absolute_link = urljoin('https://www.grantleigh.com.au/', link['href'])
                if absolute_link not in sublinks:
                    if temp in absolute_link:
                        sublinks.append(absolute_link)
                

In [4]:
print(sublinks)

['https://www.grantleigh.com.au/projects', 'https://www.grantleigh.com.au/gallery', 'https://www.grantleigh.com.au/testimonials', 'https://www.grantleigh.com.au/about-grantleigh-homes', 'https://www.grantleigh.com.au/architectural-design-drafting-servi', 'https://www.grantleigh.com.au/construction-rebuild-sydney', 'https://www.grantleigh.com.au/interior-fit-outs', 'https://www.grantleigh.com.au/renovation-extension-builder-sydney']


In [1]:
import requests
import json
import re
import pandas as pd
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from urllib.parse import urljoin
import xlsxwriter
import time
import pandas as pd

# set headers
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.5',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1'
}
all_data = []

# import website and company name from excel file
urls = pd.read_excel('yellowpages5.xlsx')
urls = urls[['name','website']]
urls['text'] = ''
urls['sublinks'] = ''
urls['sublinkstext'] = ''
# scrape website text and store in dataframe
# something wrong with index 958
with requests.Session() as s:
    for i in range(len(urls)):
        try:
            start_time_1 = time.time()
            temp = urls['website'][i].replace('https://www.','')
            r = s.get(urls['website'][i], verify=False, headers=headers)
            soup = BeautifulSoup(r.text, 'html.parser')
            text = soup.get_text()
            text = text.replace('\n', '').replace('\t', '')
            sublinks=[]
            for link in soup.find_all('a', href=True):
                absolute_link = urljoin(urls['website'][i], link['href'])
                if absolute_link not in sublinks:
                    if temp in absolute_link:
                        sublinks.append(absolute_link)
                
            urls['text'][i] = text
            urls['sublinks'][i] = sublinks
            duration_1 = time.time() - start_time_1
            for x in urls['sublinks'][i]:
                try:
                    start_time_2 = time.time()
                    t = s.get(x, verify=False, headers=headers)
                    soup1 = BeautifulSoup(t.text, 'html.parser')
                    text1 = soup1.get_text()
                    text1 = text1.replace('\n', '').replace('\t', '')
                    urls['sublinkstext'][i] = urls['sublinkstext'][i] + text1
                    duration_2 = time.time() - start_time_2
                    if duration_2 > 300:
                        raise TimeoutError
                except TimeoutError:
                    print('timeout')
                    print(x)
                    pass
                except:
                    print('error in sublink')
                    print(x)
                    pass
            
            if duration_1 > 600:
                raise TimeoutError
           
        except TimeoutError:
            print('timeout')
            print(i)
            pass
        except:
            print('error in main link')
            print(i)
            pass
    urls.to_excel('all_text_try.xlsx', engine='xlsxwriter')    

error in sublink
mailto:info@hursthomes.com.au
error in main link
15


c:\Users\htw10\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


error in sublink
http://www.jayceesconstructions.com.au/residential-concreting
error in sublink
http://www.jayceesconstructions.com.au/demolition
error in main link
55
error in main link
60
error in main link
69
error in sublink
https://malibuhomes.com.au/wp-content/uploads/2022/06/3P5A7411.jpg
error in sublink
https://malibuhomes.com.au/wp-content/uploads/2022/06/3P5A7469.jpg
error in sublink
mailto:admin@ptbuilders.com.au


,name,website,text,sublinks,sublinkstext
1,AC & EA Findlay Constructions,http://www.findlayconstructions.com.au,Findlay Constructions | Expert design and cons...,"[http://www.findlayconstructions.com.au, http:...",Findlay Constructions | Expert design and cons...


In [1]:
urls.to_excel('all_text_6.xlsx', engine='xlsxwriter')

NameError: name 'urls' is not defined